In [2]:
import open3d as o3d
import numpy as np

In [17]:
source = o3d.io.read_point_cloud("scans/A2L-D6-8-C-0_pc.ply")
target = o3d.io.read_point_cloud("scans/A2L-D6-8-C-8_pc.ply")

In [18]:
source_down = source.voxel_down_sample(voxel_size=0.02)
target_down = target.voxel_down_sample(voxel_size=0.02)
source_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
target_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

In [19]:
source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    source_down, o3d.geometry.KDTreeSearchParamHybrid(radius=0.25, max_nn=100))
target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    target_down, o3d.geometry.KDTreeSearchParamHybrid(radius=0.25, max_nn=100))

In [20]:
distance_threshold = 0.05  # Distance maximale pour considérer une correspondance
result_ransac = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source_down, target_down, source_fpfh, target_fpfh, True,
    distance_threshold,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    ransac_n=4,
    checkers=[
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    ],
    criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(max_iteration=50000, confidence=0.999)
)

In [ ]:
result_icp = o3d.pipelines.registration.registration_icp(
    source, target, 0.02, result_ransac.transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPoint()
)

: 